In [1]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, KBinsDiscretizer, RobustScaler, PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import IncrementalPCA
from tabulate import tabulate
import tensorflow as tf
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
warnings.filterwarnings("ignore", category=DeprecationWarning)

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


## Initial Loading

In [3]:

ddi_fp = "..\\Data Files\\drugbank\\drugbank.tab"
ddi = pd.read_csv(ddi_fp, sep='\t')

kaggle_fp = "..\\Data Files\\SMILES-Kaggle\\chembl_22_clean_1576904_sorted_std_final.smi"
smiles = pd.read_csv(kaggle_fp, sep='\t')

ddi["Y"] = ddi["Y"].astype("category")
ddi["Map"] = ddi["Map"].astype("category")

#counting interaction types for potential later weighting
interaction_counts = pd.DataFrame(ddi['Y'].value_counts().rename_axis('y').reset_index(name='count')).sort_values(by='count', ascending=False)
interaction_counts['row_num'] = interaction_counts.index + 1
interaction_counts['log_count'] = np.log(interaction_counts['count'])

#listing longer explanations of interaction types for later use
interaction_types = ddi[['Y','Map']].drop_duplicates(subset=['Y'])

#remove longer name of interaction type from main dataset
ddi = ddi.drop("Map",axis=1)

ddi.head(10)

,ID1,ID2,Y,X1,X2
0,DB04571,DB00460,1,CC1=CC2=CC3=C(OC(=O)C=C3C)C(C)=C2O1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
1,DB00855,DB00460,1,NCC(=O)CCC(O)=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
2,DB09536,DB00460,1,O=[Ti]=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
3,DB01600,DB00460,1,CC(C(O)=O)C1=CC=C(S1)C(=O)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
4,DB09000,DB00460,1,CC(CN(C)C)CN1C2=CC=CC=C2SC2=C1C=C(C=C2)C#N,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
5,DB11630,DB00460,1,OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
6,DB00553,DB00460,1,COC1=C2OC(=O)C=CC2=CC2=C1OC=C2,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
7,DB06261,DB00460,1,[H]N([H])CC(=O)CCC(=O)OCCCCCC,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
8,DB01878,DB00460,1,O=C(C1=CC=CC=C1)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
9,DB00140,DB00460,1,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...


In [6]:
ddi.shape

(191808, 5)

In [7]:
#quick function to turn a list of size 1 lists of strings into a list of strings, for later use
def delist(list_of_lists):
    list_of_strings = []
    for inner_list in list_of_lists:
        string = inner_list[0]
        list_of_strings.append(string)
    return list_of_strings

## Preprocessing

In [9]:

# counting drugs by number of mentions in database
old = pd.DataFrame()
old["total"] = ddi['ID1'].value_counts()
old = old.reset_index()
old.columns = ['ID', 'count'] 
new = pd.DataFrame()
new["total"] = ddi['ID2'].value_counts()
new = new.reset_index()
new.columns = ['ID', 'count'] 
drug_counts = pd.merge(old,new,how='outer',on='ID').fillna(0)
drug_counts['total'] = drug_counts['count_x'] + drug_counts['count_y']

drug_counts = drug_counts.sort_values(by='total')
drug_counts_one = pd.DataFrame(drug_counts[drug_counts['total']==1]['ID'])

#removing drugs only in database once
ddi_proc = ddi[ ~ddi['ID1'].isin(drug_counts_one['ID'])]
ddi_proc = ddi_proc[ ~ddi_proc['ID2'].isin(drug_counts_one['ID'])]

#removing one particular drug with a problematic SMILES code
ddi_proc = ddi_proc[ddi_proc['X1']!="OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1"]


<>:21: SyntaxWarning: invalid escape sequence '\C'
<>:21: SyntaxWarning: invalid escape sequence '\C'
C:\Users\warre\AppData\Local\Temp\ipykernel_10976\3570682957.py:21: SyntaxWarning: invalid escape sequence '\C'
  ddi_proc = ddi_proc[ddi_proc['X1']!="OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1"]


In [10]:
#create main datasets

data = dc.data.NumpyDataset(X=ddi_proc[['X1','X2']], y=ddi[['Y']])
df = data.to_dataframe()
df = df.sample(frac=1).reset_index(drop=True)

X_one = delist(df[["X1"]].values.tolist())
X_two = delist(df[["X2"]].values.tolist())

search_string = "nan"

count = X_one.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_one.")
count = X_two.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_two.")

In [11]:
# reduce dataset down to equal number of top 20 categories to create an equalized dataset

top20 = interaction_counts[interaction_counts['row_num'] <= 20]

reduced_df = df.merge(top20['y'], on='y')
reduced_df=reduced_df.sample(frac=1).reset_index(drop=True)

eq_df = reduced_df.groupby('y').apply(lambda x: x.sample(min(len(x), 500))).reset_index(drop=True)

eq_df

,X1,X2,y,w,ids
0,CC1=CC=C(NC(=O)C2(CC2)C2=CC=C3OC(F)(F)OC3=C2)N...,ClC1=CC2=C(C=C1)N1C=NN=C1CN=C2C1=CC=CC=C1,4,1.0,1140
1,OP(O)(=O)OCN1C(=O)NC(C1=O)(C1=CC=CC=C1)C1=CC=C...,[H][C@@]1(CC[C@]2(O)[C@]3([H])CC[C@]4([H])C[C@...,4,1.0,2472
2,CC1=CC=C(NC(=O)C2(CC2)C2=CC=C3OC(F)(F)OC3=C2)N...,CC(O)(CS(=O)(=O)C1=CC=C(F)C=C1)C(=O)NC1=CC(=C(...,4,1.0,5280
3,CCC1(C(=O)NC(=O)NC1=O)C1=CC=CC=C1,[H][C@]12CC[C@]([H])([C@H]([C@H](C1)OC(=O)C1=C...,4,1.0,902
4,CO[C@H]1\C=C\O[C@@]2(C)OC3=C(C2=O)C2=C(C(O)=C3...,CC(C)NCC(O)COC1=CC=C(COCCOC(C)C)C=C1,4,1.0,1088
...,...,...,...,...,...
9995,[H][C@@]12C[C@@H](C)[C@](OC(=O)C3=CC=CO3)(C(=O...,NS(=O)(=O)C1=CC2=C(NC(CC3=CC=CC=C3)NS2(=O)=O)C...,83,1.0,190935
9996,[H][C@@]12CC[C@](O)(C(=O)CO)[C@@]1(C)C[C@H](O)...,CN1C(CSCC(F)(F)F)NC2=CC(Cl)=C(C=C2S1(=O)=O)S(N...,83,1.0,190503
9997,[H][C@@]12C[C@@H](C)[C@H](C(=O)CO)[C@@]1(C)C[C...,NS(=O)(=O)C1=C(Cl)C=C2NC=NS(=O)(=O)C2=C1,83,1.0,190954
9998,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...,CCC1NC(=O)C2=CC(=C(Cl)C=C2N1)S(N)(=O)=O,83,1.0,190575


In [12]:
reduced_df.shape

(178494, 5)

## Featurization and Dimensionality Reduction

In [14]:
#featurize using circular fingerprint

cf_featurizer = dc.feat.CircularFingerprint()

def circular_fingerprint(smiles):
    try:
        mol = cf_featurizer(smiles)
        return mol
    except Exception as e:
        print(f"Error fingerprinting {smiles}: {e}")
        return None  # Skipping invalid SMILES


#other way to featurize a molecule
#cm_featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

#example process with just 10000 rows
df_example = df.head(10000)
df_example = df_example.reset_index()
df_example.rename(columns={'index':'col'},  inplace=True)

df_example_cf = pd.DataFrame()
df_example_cf['x1_cf'] = df_example['X1'].apply(circular_fingerprint)
df_example_cf['x2_cf'] = df_example['X2'].apply(circular_fingerprint)
df_example_cf['col'] = df_example_cf.index
df_example_cf.dropna(subset=['x1_cf', 'x2_cf'], inplace=True)


df_example_x1_cf = pd.DataFrame(delist(df_example_cf['x1_cf']))
df_example_x1_cf.rename(columns=lambda x: "x1_cf_"+str(x+1), inplace=True)
df_example_x1_cf['col'] = df_example_cf['col']

df_example_x2_cf = pd.DataFrame(delist(df_example_cf['x2_cf']))
df_example_x2_cf.rename(columns=lambda x: "x2_cf_"+str(x+1), inplace=True)
df_example_x2_cf['col'] = df_example_cf['col']


df_example = df_example.merge(df_example_x1_cf, on="col")
df_example = df_example.merge(df_example_x2_cf, on="col")
df_example.dropna(inplace=True)

df_example

[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerator
[23:04:53] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerator
[23:04:57] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerator
[23:05:04] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerator
[23:05:41] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerator
[23:05:48] DEPRECATION WARNING: please use MorganGenerat

,col,X1,X2,y,w,ids,x1_cf_1,x1_cf_2,x1_cf_3,x1_cf_4,...,x2_cf_2039,x2_cf_2040,x2_cf_2041,x2_cf_2042,x2_cf_2043,x2_cf_2044,x2_cf_2045,x2_cf_2046,x2_cf_2047,x2_cf_2048
0,0,[H][C@]1(CCCN1C(=O)[C@@H](NC(=O)OC)C(C)C)C1=NC...,C[C@@H](O[C@H]1OCCN(CC2=NNC(=O)N2)[C@H]1C1=CC=...,73,1.0,175143,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,CCN(CC)C(=O)NC1=CC=C(OCC(O)CNC(C)(C)C)C(=C1)C(...,C[C@@H](OC1=CC(=CN=C1N)C1=CN(N=C1)C1CCNCC1)C1=...,47,1.0,64982,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,CC1=CC(=NO1)C(=O)NNCC1=CC=CC=C1,CC1CC2=CC=CC=C2N1NC(=O)C1=CC(=C(Cl)C=C1)S(N)(=...,60,1.0,140284,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,CS(=O)(=O)NC1=C(OC2=CC=CC=C2)C=C(C=C1)[N+]([O-...,OC(=O)C1=C(O)C=CC(=C1)C1=C(F)C=C(F)C=C1,49,1.0,120296,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,[H][C@@](C)(C(O)=O)C1=CC(=CC=C1)C(=O)C1=CC=CC=C1,CC1=NS(=O)(=O)C2=C(N1)C=CC(Cl)=C2,49,1.0,82109,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,9993,CCCCC1C(=O)N(N(C1=O)C1=CC=C(O)C=C1)C1=CC=CC=C1,[H][C@@]12CC[C@](O)(C(=O)CO)[C@@]1(C)CC(=O)[C@...,49,1.0,78184,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9992,9994,[Cl-].CN(C)C1=CC2=[S+]C3=C(C=CC(=C3)N(C)C)N=C2...,CC(C)COCC(CN(CC1=CC=CC=C1)C1=CC=CC=C1)N1CCCC1,60,1.0,141255,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9993,9995,CC1=CN(C(=O)C=C1)C1=CC=CC=C1,CCC(=O)NS(=O)(=O)C1=CC=C(C=C1)C1=C(C)ON=C1C1=C...,49,1.0,105794,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9994,9996,COC1=CC(NC2=C(C=NC3=CC(OCCCN4CCN(C)CC4)=C(OC)C...,CN1CCN2C(C1)C1=CC=CC=C1CC1=C2N=CC=C1,73,1.0,169723,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#Basic dimensionality reduction down to 100 components from 4096

df_X = df_example.iloc[:,6:]
df_Y = to_categorical(df_example["y"])

n_components = 100
batch_size = 1000

ipca = IncrementalPCA(n_components=n_components)

for i in range(0, df_X.shape[0], batch_size):
    X_batch = df_X[i:i + batch_size]
    ipca.partial_fit(X_batch)

X_transformed = ipca.transform(df_X)

#prove that X's shape has changed
#print("Original shape:", df_X.shape)
#print("Transformed shape:", X_transformed.shape)

df_X_proc = pd.DataFrame(X_transformed)


## Initial basic model implementation

In [17]:
#basic/rough neural network implementation

def calc_layers(X_size, Y_size):
    layers = [X_size+1]
    layer = 2
    while layer <= X_size:
        layer = int(layer * 2)
    layers.append(layer)
    if X_size > Y_size:
        while layer / 2 > Y_size and layer > 2:
            layer = layer / 2
            layers.append(int(layer))
    layers.append(Y_size)
    return layers
        
def DNN(X, Y, Epochs, batchsize, layernum=1, verbose=False):

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=27)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #define layers and nodes in each layer
    input = len(X.columns)
    output = Y.shape[1]
    if layernum=='many':
        layers = calc_layers(input,output)
    elif type(layernum)==int:
        layers = [input]
        for i in range(layernum, 1, -1):
            layer = int(round((i * (input + output) / (layernum+1)), 0))
            if layer > output:
                layers.append(layer)
        layers.append(output)
    else:
        print(f"incorrect layernum {layernum}")
        return None

    model = keras.models.Sequential()

    model.add(Dense(layers[0], activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    for layer_size in layers[1:-1]:
        model.add(Dense(layer_size, activation='relu'))
        model.add(keras.layers.Dropout(0.2))
        
    model.add(Dense(layers[-1], activation='softmax'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy','AUC','precision','recall'])
    
    model.fit(X_train, y_train, epochs=Epochs, batch_size=batchsize, validation_split=0.1, verbose=verbose)
    
    loss, accuracy, AUC, precision, recall = model.evaluate(X_test, y_test, verbose=0)
    if verbose:
        print(f"Test Loss: {loss}")
        print(f"Test Accuracy: {accuracy}")
        print(f"Test AUC: {AUC}")
        print(f"Test Precision: {precision}")
        print(f"Test Recall: {recall}")
    return model, accuracy

In [18]:
dr_model, acc = DNN(df_X_proc, df_Y, 5, 1000, 1)

In [19]:
#comparison to without Dimensionality reduction

full_model, acc = DNN(df_X, df_Y, 5, 1000, 1)

#calc_layers(len(df_X.iloc[:,1:].columns), df_Y.shape[1])

This demonstrates that dimensionality reduction is not productive to a neural network in this case.

## Adding in other variables

In [22]:
#code from Beth Farr
def compute_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return None

        # Computing Molecular Descriptors
        mol_wt = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        h_donors = Descriptors.NumHDonors(mol)
        h_acceptors = Descriptors.NumHAcceptors(mol)
        tpsa = Descriptors.TPSA(mol)
        return [mol_wt, logp, h_donors, h_acceptors, tpsa]

    except Exception as e:
        print(f"Error computing features for {smiles}: {e}")
        return None  # Skipping invalid SMILES

# Using function to X1 and X2 to extract features
df_feat = df_example
df_feat['features_X1'] = df_example['X1'].apply(compute_features)
df_feat['features_X2'] = df_example['X2'].apply(compute_features)
df_feat = df_feat.dropna(subset=['features_X1', 'features_X2'])
features_X1_df = pd.DataFrame(df_feat['features_X1'].tolist(), columns=['MolWt_X1', 'LogP_X1', 'NumHDonors_X1', 'NumHAcceptors_X1', 'TPSA_X1'])
features_X1_df['col'] = df_feat['col']
features_X2_df = pd.DataFrame(df_feat['features_X2'].tolist(), columns=['MolWt_X2', 'LogP_X2', 'NumHDonors_X2', 'NumHAcceptors_X2', 'TPSA_X2'])
features_X2_df['col'] = df_feat['col']

df_feat = pd.merge(df_feat, pd.merge(features_X1_df, features_X2_df,on='col'), on='col')
df_feat = df_feat.drop(['X1', 'X2', 'w', 'features_X1', 'features_X2'], axis=1)

df_feat_X = df_feat.iloc[:,3:]
df_feat_Y = to_categorical(df_feat["y"])

feat_model, acc = DNN(df_feat_X, df_feat_Y, 5, 1000, 1)


[23:09:46] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[23:09:46] SMILES Parse Error: check for mistakes around position 76:
[23:09:46] C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C
[23:09:46] ~~~~~~~~~~~~~~~~~~~~^
[23:09:46] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'


## Adding In GNN

In [24]:
#Creating GNN Model
class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

#Using PyTorch Geometric Graph Format
def create_graph_features(features, labels):
    num_nodes = len(features)

    # creating the nodes
    edge_index = torch.tensor(
        np.array([[i, i] for i in range(num_nodes)]).T, dtype=torch.long
    )

    graphs = []
    for i in range(len(features)):
        x = torch.tensor(features[i], dtype=torch.float).unsqueeze(0)
        y = torch.tensor([labels[i]], dtype=torch.long)
        graph = Data(x=x, edge_index=edge_index, y=y)
        graphs.append(graph)
    return graphs

#Training the GNN Model
def GNN_train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(torch.device("cpu"))  # Ensure correct device
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.squeeze(1), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

#Evaluating the Model
def GNN_test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(torch.device("cpu"))  # Ensure correct device
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y).sum().item()
            total += data.y.cpu().size(0)
    acc = correct / total
    return acc

def perform_GNN(X, Y, epochs, batchsize, verbose=False):

    #Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
                                                    
    train_graphs = create_graph_features(X_train, y_train)
    test_graphs = create_graph_features(X_test, y_test)

    # Creating data loaders
    train_loader = DataLoader(train_graphs, batch_size=batchsize, shuffle=True)
    test_loader = DataLoader(test_graphs, batch_size=batchsize, shuffle=False)

    #Initialising the Model
    input_dim = X.shape[1]
    output_dim = len(np.unique(Y))
    hidden_dim = int(round((input_dim + output_dim)/2,0))
    
    model = GNN(input_dim, hidden_dim, output_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    #Running Training
    accuracy = 0
    for epoch in range(epochs):
        loss = GNN_train(model, train_loader, optimizer, criterion)
        acc = GNN_test(model, test_loader)
        if verbose:
            print(f"Epoch {epoch+1}: Loss={loss:.4f}, Test Accuracy={acc:.4f}")
        if acc > accuracy:
            accuracy = acc

    #Saving the model for future use
    torch.save(model.state_dict(), "gnn_model.pth")
    return model, accuracy
    

In [25]:
label_encoder = LabelEncoder()
graph_y = label_encoder.fit_transform(df_feat['y'])

model_gnn = perform_GNN(df_feat_X.values, graph_y, 5, 1000)

In [26]:
def ddi_featurize(df, X1_name, X2_name):
    df_cf = df
    df_cf['x1_cf'] = df[X1_name].apply(circular_fingerprint)
    df_cf['x2_cf'] = df[X2_name].apply(circular_fingerprint)
    df_cf['col'] = df_cf.index
    df_cf.dropna(subset=['x1_cf', 'x2_cf'], inplace=True)
    
    df_x1_cf = pd.DataFrame(delist(df_cf['x1_cf']))
    df_x1_cf.rename(columns=lambda x: "x1_cf_"+str(x+1), inplace=True)
    df_x1_cf['col'] = df_cf['col']
    
    df_x2_cf = pd.DataFrame(delist(df_cf['x2_cf']))
    df_x2_cf.rename(columns=lambda x: "x2_cf_"+str(x+1), inplace=True)
    df_x2_cf['col'] = df_cf['col']

    print(df_cf.columns)
    df_temp = df_cf.drop(['x1_cf', 'x2_cf'], axis=1)
    df_temp = df_temp.merge(df_x1_cf, on="col")
    df_temp = df_temp.merge(df_x2_cf, on="col")
    df_temp.dropna(inplace=True)

    df_temp['features_X1'] = df_temp['X1'].apply(compute_features)
    df_temp['features_X2'] = df_temp['X2'].apply(compute_features)
    df_temp = df_temp.dropna(subset=['features_X1', 'features_X2'])
    features_X1_df = pd.DataFrame(df_temp['features_X1'].tolist(), columns=['MolWt_X1', 'LogP_X1', 'NumHDonors_X1', 'NumHAcceptors_X1', 'TPSA_X1'])
    features_X1_df['col'] = df_temp['col']
    features_X2_df = pd.DataFrame(df_temp['features_X2'].tolist(), columns=['MolWt_X2', 'LogP_X2', 'NumHDonors_X2', 'NumHAcceptors_X2', 'TPSA_X2'])
    features_X2_df['col'] = df_temp['col']
    df_temp.drop(['features_X1', 'features_X2'], axis=1, inplace = True)
    
    df_final = pd.merge(df_temp, pd.merge(features_X1_df, features_X2_df,on='col'), on='col')
    df_final.drop(['col'], axis=1, inplace = True)

    return df_final

In [27]:
eq_df['col'] = eq_df.index
eq_df_feat = ddi_featurize(eq_df, "X1","X2")
eq_df_feat = eq_df_feat.drop(['X1', 'X2', 'w'], axis=1)

[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerator
[23:10:58] DEPRECATION WARNING: please use MorganGenerat

Index(['X1', 'X2', 'y', 'w', 'ids', 'col', 'x1_cf', 'x2_cf'], dtype='object')


In [28]:
eq_model_dnn, eq_dnn_acc = DNN(eq_df_feat.iloc[:,2:], to_categorical(eq_df_feat['y']), 5, 1000, 1)

In [29]:
eq_graph_y = label_encoder.fit_transform(eq_df_feat['y'])

eq_model_gnn, eq_gnn_acc = perform_GNN(eq_df_feat.iloc[:,2:].values, eq_graph_y, 5, 1000)

In [30]:
def opt_hps(func, X, Y, epochs, batchsizes):
    best_epochs = 0
    best_batchsize = 0
    best_acc = 0 
    grid = pd.DataFrame(columns=epochs, index=batchsizes)
    for epoch_num in epochs:
        for batchsize in batchsizes:
            model, acc = func(X, Y, epoch_num, batchsize)
            grid.loc[[batchsize][epoch_num]] = acc
            if acc > best_acc:
                best_epochs = epoch_num
                best_batchsize = batchsize
                best_acc = acc
    print(f"Optimal hyperparams: {best_epochs} epochs, {best_batchsize} batchsize. Accuracy: {best_acc}")
    return grid

In [33]:
dnn_epochs = [7,8,9,10,11]
dnn_batchsizes = [150, 200, 250, 300, 350, 400]

gnn_epochs = [5,6,7,8,9]
gnn_batchsizes = [100, 125,150, 175, 200, 225]

In [35]:
dnn_grid = opt_hps(DNN, eq_df_feat.iloc[:,2:], to_categorical(eq_df_feat['y']), dnn_epochs, dnn_batchsizes)

dnn_grid

KeyboardInterrupt: 

In [ ]:
gnn_grid = opt_hps(perform_GNN, eq_df_feat.iloc[:,2:].values, eq_graph_y, my_epochs, my_batchsizes)

gnn_grid